### Organiza o resultado do algoritmo apriori
Após executar o algoritmo apriori, o resultado sofre algumas modificações facilitando sua visualização.<br>
Remoção de parte da string desnecessária no arquivo resultante do algoritmo apriori.

# Bibliotecas

In [47]:
import pandas as pd
from decimal import Decimal

# Carregamento dos dados

In [48]:
# Nome do arquivo com o resultado apriori
nomeArq = 'AprioriMlxtend_ES_datatran2019_Apriori'

In [49]:
data = pd.read_csv(nomeArq +'.csv', encoding='ISO-8859-1', sep=';')

In [50]:
data.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'ilesos=0', 'classif...",0.130542,0.286066,0.130542,1.0,3.495695,0.093198,inf,3,3
1,"frozenset({'pessoas=1', 'br=101.0', 'classific...","frozenset({'ignorados=0', 'feridos=1', 'ilesos...",0.130542,0.183673,0.130542,1.0,5.444444,0.106565,inf,3,3
2,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'mortos=0', 'classif...",0.130542,0.732231,0.130542,1.0,1.365690,0.034955,inf,3,3
3,"frozenset({'pessoas=1', 'br=101.0', 'classific...","frozenset({'ignorados=0', 'feridos=1', 'mortos...",0.130542,0.525686,0.130542,1.0,1.902276,0.061918,inf,3,3
4,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'veiculos=1', 'class...",0.130542,0.294511,0.130542,1.0,3.395460,0.092096,inf,3,3


In [51]:
data.shape

(2837, 11)

In [52]:
data.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'antecedent_len', 'consequent_len'],
      dtype='object')

# Organização dos dados

## Remoção de parte da string desnecessária (antecedents, consequents)

In [53]:
tamLinha = data.shape[0]

In [54]:
Ant_Suc = []
Ant_Suc.append('antecedents')
Ant_Suc.append('consequents')
# Ant_Suc

In [55]:
for c in Ant_Suc:
    for i in range(tamLinha):
        # Remoção do 'frozenset(' e ')'
        data.at[i, c] = data.at[i, c].replace("frozenset(", "")
        data.at[i, c] = data.at[i, c].replace(")", "")
        
        # Remoção do 'nan'
        data.at[i, c] = data.at[i, c].replace("'nan'", "")
        data.at[i, c] = data.at[i, c].replace("{}", "")
        
        # Remoção de alguns casos depois da remoção do 'nan'
        data.at[i, c] = data.at[i, c].replace(" , ", " ")
        data.at[i, c] = data.at[i, c].replace(",}", "}")
        data.at[i, c] = data.at[i, c].replace(", }", "}")
        data.at[i, c] = data.at[i, c].replace("{, ", "{")

## Remoção das linhas vazias

In [56]:
data.shape

(2837, 11)

In [57]:
# Remoção de linhas com antecedentes ou consequenes nulos
df_remove = data.loc[((data['antecedents'] == '') | (data['consequents'] == ''))]
data = data.drop(df_remove.index)

In [58]:
data.shape

(2837, 11)

## Removendo Linhas duplicadas

In [59]:
data.shape

(2837, 11)

In [60]:
data.drop_duplicates(inplace=True) # Verifica a existência de linhas duplicadas

In [61]:
# Ordena de acordo com a coluna support de forma decrescente
data.sort_values(by=['confidence', 'support'], inplace=True, ascending=False)

# Remoção de regras similares

In [62]:
# data.sort_values(by=['support'], ascending=False, inplace=True)

In [63]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
data.to_csv(nomeArq + '_Backup.csv', encoding= 'ISO-8859-1', sep= ';', index=None)
data = pd.read_csv(nomeArq +'_Backup.csv', encoding='ISO-8859-1', sep=';')

In [64]:
def asterix(num):
    a = '*'
    x =''
    for i in range(num):
        x = x + a    
    return x

In [65]:
%%time
tamLinha = data.shape[0]
flag = 1
listRemove = []
for i in range(0, tamLinha):
    flag = 1
    for j in range(i+1, tamLinha):
        if ( (data.at[i, 'antecedents'] == data.at[j, 'consequents']) and (data.at[i, 'consequents'] == data.at[j, 'antecedents'])): # testar equals
                count = i+2 
                data.at[j, 'antecedents'] = str(count) + (asterix(flag))
                data.at[j, 'consequents'] = str(count) + (asterix(flag))
#                 data.at[j, 'antecedents'] = ''
#                 data.at[j, 'consequents'] = ''
                listRemove.append(j) # Adiciona em uma lista as linhas que serão removidas logo em seguida
                flag = flag + 1

Wall time: 1min 17s


In [66]:
# A lista possui o número das linhas que possuem dados repetidos e serão removidas
for r in listRemove:
    print("Linha:", r)
    data.drop(r, inplace=True)

print("Nº de linhas removidas: ", len(listRemove))

Linha: 62
Linha: 1124
Linha: 1123
Linha: 1122
Linha: 1188
Linha: 1491
Linha: 1652
Linha: 1651
Linha: 1650
Linha: 1647
Linha: 1621
Linha: 1620
Linha: 1619
Nº de linhas removidas:  13


In [67]:
data.shape

(2824, 11)

# Limitar casas decimais

In [68]:
decimals = 8

listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    data[i] = data[i].apply(lambda x: round(x, decimals))
#     data[i]= pd.to_numeric(data[i], downcast='float')

In [69]:
# listCol = ['confidence', 'lift', 'leverage', 'conviction']
# for i in listCol:
#     for j in range(data.shape[0]):
#         data.at[j, i] = Decimal(data.at[j, i])
# #         data.at[j, i] = float(data.at[j, i])
#         data.at[j, i] = "{:.3f}".format(data.at[j, i])

# Incluir uma nova coluna com a identificação de cada linha

In [70]:
#Célula utilizada para atualizar a linha do dataframe. Iniciando a linha em 0.
# Salvando em um arquivo .csv
data.to_csv(nomeArq + '_Backup.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
data = pd.read_csv(nomeArq + '_Backup.csv', encoding='ISO-8859-1', sep=';')

In [71]:
# Cria uma nova coluna que servirá para identificar cada regra de associação com um número inteiro.
idx = 0
new_col = 0
data.insert(loc=idx, column='n_row', value=new_col)

row =1
for i in range(data.shape[0]):
    data.at[i, 'n_row'] = int(row)
    row = row + 1

# Resultado final

In [72]:
data.head(10)

,n_row,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,1,"{'pessoas=2', 'feridos=1', 'ilesos=1'}","{'ignorados=0', 'mortos=0', 'classificacao_aci...",0.267065,0.732231,0.267065,1.0,1.365690,0.071512,inf,3,3
1,2,"{'pessoas=2', 'ilesos=1', 'classificacao_acide...","{'ignorados=0', 'feridos=1', 'mortos=0'}",0.267065,0.525686,0.267065,1.0,1.902276,0.126673,inf,3,3
2,3,"{'pessoas=2', 'feridos=1', 'ilesos=1', 'veicul...","{'ignorados=0', 'mortos=0', 'classificacao_aci...",0.227657,0.732231,0.227657,1.0,1.365690,0.060959,inf,4,3
3,4,"{'pessoas=2', 'ilesos=1', 'veiculos=2', 'class...","{'ignorados=0', 'feridos=1', 'mortos=0'}",0.227657,0.525686,0.227657,1.0,1.902276,0.107981,inf,4,3
4,5,"{'feridos=1', 'pessoas=2', 'br=101.0', 'ilesos...","{'ignorados=0', 'mortos=0', 'classificacao_aci...",0.195285,0.732231,0.195285,1.0,1.365690,0.052291,inf,4,3
5,6,"{'pessoas=2', 'br=101.0', 'ilesos=1', 'classif...","{'ignorados=0', 'feridos=1', 'mortos=0'}",0.195285,0.525686,0.195285,1.0,1.902276,0.092626,inf,4,3
6,7,"{'feridos=1', 'mortos=0', 'pessoas=1'}","{'ignorados=0', 'ilesos=0', 'classificacao_aci...",0.181210,0.286066,0.181210,1.0,3.495695,0.129372,inf,3,3
7,8,"{'pessoas=1', 'feridos=1', 'classificacao_acid...","{'ignorados=0', 'mortos=0', 'ilesos=0'}",0.181210,0.286066,0.181210,1.0,3.495695,0.129372,inf,3,3
8,9,"{'ignorados=0', 'feridos=1', 'pessoas=1'}","{'mortos=0', 'ilesos=0', 'classificacao_aciden...",0.181210,0.346587,0.181210,1.0,2.885279,0.118405,inf,3,3
9,10,"{'feridos=1', 'ilesos=0', 'pessoas=1'}","{'ignorados=0', 'mortos=0', 'classificacao_aci...",0.181210,0.732231,0.181210,1.0,1.365690,0.048523,inf,3,3


# Salvando o arquivo resultante

In [73]:
data.to_csv(nomeArq + '_Mod.csv', encoding= 'ISO-8859-1', sep= ';', index=None)

In [74]:
print("ACABOU!")

ACABOU!
